### rPPG detector (GRGB)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import csv

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

loop_video = False # set to TRUE for demonstration purpose

file = 'p1_120'
filetype = '.mp4'
cap = cv2.VideoCapture('_video/' + file + filetype) 
#cap = cv2.VideoCapture('data/ground_truth/subject04/' + file + filetype) 
#cap = cv2.VideoCapture(0) # webcam

# Define region of interest: left cheek, right cheek, forehead
rois = [
    [118, 119, 100, 126, 209, 49, 129, 203, 205, 50],
    [347, 348, 329, 355, 429, 279, 358, 423, 425, 280], 
    [107, 66, 69, 109, 10, 338, 299, 296, 336, 9]] 
rppg_timeseries = []


"""
rPPG Calculation
"""
# Calculates the average RGB color of a specific ROI, by using a boundary mask defined by landmarks
def calculate_average_rgb_roi(roi_num):
    mask = np.zeros((height, width), dtype=np.uint8)
    points = np.array([[int(facial_landmarks.landmark[idx].x * width), int(facial_landmarks.landmark[idx].y * height)] for idx in rois[roi_num]])
    cv2.fillPoly(mask, [points], 255)
    mean_val = cv2.mean(image, mask=mask)
    return (mean_val[2], mean_val[1], mean_val[0])  # return R, G, B

# Calculates the average RGB color value of multiple ROIs
def calculate_average_rgb():
    roi_list = []
    for i in range(0, len(rois)):
        roi_list.append(calculate_average_rgb_roi(i))
    sum_r, sum_g, sum_b = 0, 0, 0
    num_values = len(roi_list) # number of RGB tuples in the list
    for rgb in roi_list:
        sum_r += rgb[0]
        sum_g += rgb[1]
        sum_b += rgb[2]
    avg_r = sum_r / num_values
    avg_g = sum_g / num_values
    avg_b = sum_b / num_values
    return avg_r, avg_g, avg_b

# Calculates the rPPG signal from RGB
def calculate_rppg():
    rgb = calculate_average_rgb()
    rppg = rgb[1] / rgb[0] + rgb[1] / rgb[2] # GRGB = G/R + G/B
    return rppg


"""
Video Display and rPPG Export
"""
while True:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        break
    ret, image = cap.read()
    if not ret:
        if loop_video:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
            ret, image = cap.read() # reads a new frame
        else: # ends video playback
            cap.release()
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break

    height, width, _ = image.shape
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # mediapipe needs RGB instead of BGR color space
    result = face_mesh.process(rgb_image)

    # Display facial landmarks and raw numerical value of rPPG signal:
    if result.multi_face_landmarks:
        for facial_landmarks in result.multi_face_landmarks:
            for r in range(len(rois)): # for every ROI
                for i in rois[r]: # for every landmark in ROI
                    pt = facial_landmarks.landmark[i]
                    x = int(pt.x * width) # normalized to image coordinates
                    y = int(pt.y * height)
                    cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
        rppg_value = calculate_rppg()
        cv2.putText(image, 'GRGB rPPG: ' + str(rppg_value), (20, 30), 2, 1, (0, 255, 0))
        rppg_timeseries.append(rppg_value)
    else:
        cv2.putText(image, 'No face detected', (20, 30), 2, 1, (0, 0, 255))
        rppg_timeseries.append(np.nan)

    # Display video image
    cv2.imshow('rPPG detector', image)
    cv2.waitKey(1) # waiting and going to the next frame

# Save rPPG signal to csv after video playback
with open('_export/rppg_' + file + '.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(rppg_timeseries)
